<a href="https://colab.research.google.com/github/AnthonyAugust/Projetos-Finalizados/blob/Projetos-Finalizados/Assist%C3%AAncia_Virtual_(Comando_de_Voz_%2B_Texto).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install gTTS SpeechRecognition pydub wikipedia
!apt-get -qq install -y ffmpeg > /dev/null

In [1]:
import os, re, urllib.parse
from gtts import gTTS
from IPython.display import Audio, HTML, display, Javascript, clear_output
from pydub import AudioSegment
import speech_recognition as sr
import wikipedia

# Wikipedia em português
wikipedia.set_lang("pt")

def speak_pt(texto, filename="tts.mp3"):
    """Converte texto em fala (PT-BR) e toca no Colab."""
    tts = gTTS(text=texto, lang="pt", tld="com.br")
    tts.save(filename)
    display(Audio(filename, autoplay=True))

def abrir_link(url, rotulo=None):
    """Mostra um link clicável que abre em nova aba."""
    rotulo = rotulo or url
    display(HTML(f'<a href="{url}" target="_blank" rel="noopener noreferrer">{rotulo}</a>'))

def limpar_texto(s):
    """Normaliza texto para facilitar matching de comandos."""
    return re.sub(r"\s+", " ", s).strip().lower()

In [ ]:
#Testando o TTS
speak_pt("Olá! Eu já consigo falar. Vamos continuar a configuração.")

In [2]:
from base64 import b64decode
from google.colab import output

JS_GRAVAR = r"""
async function gravarAudio(segundos) {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const rec = new MediaRecorder(stream);
  let dados = [];

  rec.ondataavailable = e => dados.push(e.data);
  rec.start();

  await new Promise(r => setTimeout(r, segundos * 1000));
  rec.stop();

  await new Promise(resolve => rec.onstop = resolve);
  const blob = new Blob(dados, { type: 'audio/webm' });
  const arrayBuffer = await blob.arrayBuffer();
  const base64 = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)));
  return base64;
}
"""

def gravar_para_wav(segundos=5, base="entrada"):
    """Grava 'segundos' de áudio do mic e retorna o caminho do WAV."""
    display(Javascript(JS_GRAVAR))
    b64 = output.eval_js(f"gravarAudio({int(segundos)})")

    webm_path = f"/content/{base}.webm"
    wav_path  = f"/content/{base}.wav"

    with open(webm_path, "wb") as f:
        f.write(b64decode(b64))

    # Converte WEBM -> WAV (16kHz mono é suficiente p/ STT)
    audio = AudioSegment.from_file(webm_path, format="webm")
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(wav_path, format="wav")
    return wav_path


In [ ]:
#Testando a gravação (Fale por 3 segundos e ouça)
wav_teste = gravar_para_wav(3, base="teste_mic")
display(Audio(wav_teste, autoplay=True))

In [3]:
def transcrever_pt(caminho_wav):
    """Transcreve áudio em PT-BR. Retorna string com o texto reconhecido."""
    r = sr.Recognizer()
    with sr.AudioFile(caminho_wav) as source:
        audio = r.record(source)
    try:
        texto = r.recognize_google(audio, language="pt-BR")
        return texto
    except sr.UnknownValueError:
        return ""
    except sr.RequestError as e:
        # Falha na API (sem internet, limite, etc.)
        print("Erro no serviço de reconhecimento:", e)
        return ""



In [ ]:
# Teste: grave 4s, depois transcreva
wav = gravar_para_wav(4, base="fala_teste")
print("Transcrição:", transcrever_pt(wav))

In [4]:
def comando_wikipedia(frase):
    # tenta extrair o tema depois de 'wikipedia' ou de 'pesquisar ... na wikipedia'
    tema = None

    m = re.search(r"(?:wikip[eé]dia?\s*(?:sobre|de)?\s*(.+))", frase)
    if m:
        tema = m.group(1).strip()
    else:
        m = re.search(r"pesquisar\s+(.+?)\s+na\s+wikip[eé]dia?", frase)
        if m:
            tema = m.group(1).strip()

    if not tema:
        speak_pt("Qual assunto você quer pesquisar na Wikipédia?")
        return

    try:
        resumo = wikipedia.summary(tema, sentences=2, auto_suggest=True, redirect=True)
    except wikipedia.DisambiguationError as e:
        # Se houver várias páginas, tenta a primeira opção
        opcao = e.options[0]
        resumo = wikipedia.summary(opcao, sentences=2, auto_suggest=True, redirect=True)
        tema = opcao
    except wikipedia.PageError:
        speak_pt("Não encontrei esse assunto na Wikipédia.")
        return

    print(f"🔎 Wikipédia — {tema}\n\n{resumo}")
    speak_pt(resumo)

def comando_youtube(frase):
    # pega o que vier depois de "youtube" ou de "abrir youtube para ..."
    m = re.search(r"(?:abrir\s+)?youtube(?:\s+para)?\s+(.+)", frase)
    termo = m.group(1).strip() if m else ""
    if not termo:
        speak_pt("Abrindo YouTube. Diga um termo depois de 'YouTube' para buscar algo específico.")
        abrir_link("https://www.youtube.com", "Abrir YouTube")
        return

    q = urllib.parse.quote(termo)
    url = f"https://www.youtube.com/results?search_query={q}"
    speak_pt(f"Aqui estão resultados no YouTube para {termo}.")
    abrir_link(url, f"Abrir YouTube: {termo}")

def comando_farmacia(frase):
    # Link que utiliza sua localização do navegador para listar farmácias próximas
    url = "https://www.google.com/maps/search/farmacia+perto+de+mim"
    speak_pt("Mostrando farmácias próximas.")
    abrir_link(url, "Abrir farmácias próximas no Google Maps")

def executar_comando(frase_original):
    frase = limpar_texto(frase_original)

    if any(p in frase for p in ["sair", "encerrar", "tchau"]):
        speak_pt("Até mais! Encerrando.")
        return "SAIR"

    if "wikip" in frase or "pesquisar" in frase and "wikip" in frase:
        comando_wikipedia(frase)
        return

    if "youtube" in frase:
        comando_youtube(frase)
        return

    if "farm" in frase or "farmácia" in frase or "perto" in frase and "farm" in frase:
        comando_farmacia(frase)
        return

    # ajuda
    if any(p in frase for p in ["ajuda", "help", "comandos"]):
        msg = ("Você pode dizer: "
               "‘pesquisar [termo] na wikipedia’, "
               "‘wikipedia sobre [termo]’, "
               "‘abrir youtube para [termo]’, "
               "ou ‘farmácia mais próxima’. "
               "Diga ‘sair’ para encerrar.")
        print(msg)
        speak_pt(msg)
        return

    # Se não entendeu:
    msg = "Não entendi o comando. Diga 'ajuda' para ver exemplos."
    print(msg)
    speak_pt(msg)

In [5]:
import threading

# Função para TTS assíncrona (não bloqueia o fluxo)
def speak_pt_async(texto, filename="tts.mp3"):
    def _speak():
        tts = gTTS(text=texto, lang="pt", tld="com.br")
        tts.save(filename)
        display(Audio(filename, autoplay=True))
    thread = threading.Thread(target=_speak)
    thread.start()

In [9]:
#Assistente de busca por voz
def assistente_por_voz_colab():
    speak_pt("Assistente iniciado. Diga um comando ou 'sair' para encerrar.")

    while True:
        try:
            # Aqui você grava enquanto fala
            wav = gravar_para_wav(segundos=5, base="entrada_usuario")
            texto = transcrever_pt(wav)

            if not texto:
                print("Não captei nada. Vamos tentar de novo.")
                continue

            print(f"🗣️  Você disse: {texto}")
            resultado = executar_comando(texto)
            if resultado == "SAIR":
                break

        except Exception as e:
            print("Erro:", e)
            speak_pt("Ocorreu um erro, tentando novamente.")
assistente_por_voz_colab()


<IPython.core.display.Javascript object>

Não captei nada. Vamos tentar de novo.


<IPython.core.display.Javascript object>

🗣️  Você disse: Wikipédia
🔎 Wikipédia — a

Em tipografia, o indicador ordinal é um símbolo adjacente ao número para denotá-lo como  ordinal  (e não  cardinal). O exato símbolo utilizado varia de acordo com o idioma.


<IPython.core.display.Javascript object>

🗣️  Você disse: sair


In [6]:
#Assistente de busca por texto.
def speak_pt_colab(texto, filename="tts.mp3"):
    """Gera arquivo de áudio TTS e mostra link para tocar (não bloqueia input)."""
    tts = gTTS(text=texto, lang="pt", tld="com.br")
    tts.save(filename)
    # Link clicável para ouvir o áudio
    display(HTML(f'<a href="{filename}" target="_blank">Clique aqui para ouvir a resposta</a>'))

def assistente_por_texto_colab():
    print("Assistente de texto iniciado. Digite seu comando ou 'sair' para encerrar.")

    while True:
        cmd = input("\nDigite um comando (ou 'sair'): ").strip()
        if not cmd:
            continue  # ignora entrada vazia

        print(f"Você digitou: {cmd}")
        resultado = executar_comando(cmd)

        # Gerar TTS sem bloquear input
        speak_pt_colab(f"Resposta do assistente: {cmd}")

        if resultado == "SAIR":
            break
assistente_por_texto_colab()

Assistente de texto iniciado. Digite seu comando ou 'sair' para encerrar.

Digite um comando (ou 'sair'): 

Digite um comando (ou 'sair'): Olá
Você digitou: Olá
Não entendi o comando. Diga 'ajuda' para ver exemplos.



Digite um comando (ou 'sair'): ajuda
Você digitou: ajuda
Você pode dizer: ‘pesquisar [termo] na wikipedia’, ‘wikipedia sobre [termo]’, ‘abrir youtube para [termo]’, ou ‘farmácia mais próxima’. Diga ‘sair’ para encerrar.



Digite um comando (ou 'sair'): sair
Você digitou: sair
